In [9]:
CODE_SIZE = 64 * 64
VAL_TRUTH_PATH = "/kaggle/input/hypertension-classification/validation_truth.txt"
VAL_SET_PATH = "/kaggle/input/hypertension-classification/2. Validation Set"
TRAIN_SET_PATH = "/kaggle/input/hypertension-classification/AE Training set"
FPRATE = 0.2
EPOCHS = 30

import torch
import numpy as np

from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
import torch.nn as nn
import torch.nn.functional as F



class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        self.encoder1 = nn.Conv2d(3, 128, kernel_size=5, stride=1, padding=2)
        self.encoder2 = nn.Conv2d(128, 128, kernel_size=5, stride=1, padding=2)
        self.encoder3 = nn.Conv2d(128, 128, kernel_size=5, stride=1, padding=2)
        self.encoder4 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.encoder5 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.encoder6 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.encoder7 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=4, stride=4)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.upsample1 = nn.UpsamplingNearest2d(scale_factor=4)
        self.upsample2 = nn.UpsamplingNearest2d(scale_factor=2)
        self.decoder1 = nn.Conv2d(128, 3, kernel_size=5, stride=1, padding=2)
        self.decoder2 = nn.Conv2d(128, 128, kernel_size=5, stride=1, padding=2)
        self.decoder3 = nn.Conv2d(128, 128, kernel_size=5, stride=1, padding=2)
        self.decoder4 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.decoder5 = nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1)
        self.decoder6 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.decoder7 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        
    def encode1(self, x):
        x = F.leaky_relu(self.encoder1(x))
        x = F.leaky_relu(self.encoder2(x))
        x = self.maxpool1(x)
        return x
        
    def decode1(self, x):
        x = self.upsample1(x)
        x = F.sigmoid(self.decoder2(x))
        x = F.sigmoid(self.decoder1(x))
        return x
        
    def encode2(self, x):
        x = F.leaky_relu(self.encoder3(x))
        x = F.leaky_relu(self.encoder4(x))
        x = self.maxpool2(x)
        return x
        
    def decode2(self, x):
        x = self.upsample2(x)
        x = F.leaky_relu(self.decoder4(x))
        x = F.leaky_relu(self.decoder3(x))
        return x
    
    def encode3(self, x):
        x = F.leaky_relu(self.encoder5(x))
        x = F.leaky_relu(self.encoder6(x))
        x = self.maxpool1(x)
        x = F.leaky_relu(self.encoder7(x))
        return x
        
    def decode3(self, x):
        x = F.leaky_relu(self.decoder7(x))
        x = self.upsample1(x)
        x = F.leaky_relu(self.decoder6(x))
        x = F.leaky_relu(self.decoder5(x))
        return x
        
    def encode(self, x):
        x = self.encode1(x)
        x = self.encode2(x)
        x = self.encode3(x)
        return x
    
    def decode(self, x):
        x = self.decode3(x)
        x = self.decode2(x)
        x = self.decode1(x)
        return x

    def forward(self, x):
        x = self.encode(x)
        x = self.decode(x)
        return x




In [4]:
import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class ImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_files = os.listdir(root_dir)

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.image_files[idx])
        image = Image.open(img_name).resize((128, 128))
        if self.transform:
            image = self.transform(image)
        return image

In [5]:


"""def train(load = False):
    transform =  transforms.Compose([
        transforms.Resize((128,128)),
        transforms.ToTensor()
    ])
    dataset = ImageDataset(TRAIN_SET_PATH, transform)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    autoencoder = Autoencoder().to(device)
    if load:
        state_dict = torch.load("/kaggle/working/autoencoder1.pth")
        autoencoder.load_state_dict(state_dict)

    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.0001)
    try:
        for epoch in range(1, EPOCHS+1):
            running_loss = 0.0
            for images in tqdm(dataloader):
                images = images.to(device)
                optimizer.zero_grad()
                mids = autoencoder.encode1(images)
                outputs = autoencoder.decode1(mids)
                loss = criterion(outputs, images)
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * images.size(0)
            epoch_loss = running_loss / len(dataset)
            print(f"Epoch [{epoch}/{EPOCHS}], Loss: {epoch_loss:.8f}")
        for epoch in range(1, EPOCHS+1):
            running_loss = 0.0
            for images in tqdm(dataloader):
                images = images.to(device)
                optimizer.zero_grad()
                mids = autoencoder.encode1(images)
                outputs = autoencoder.encode2(mids)
                outputs = autoencoder.decode2(outputs)
                loss = criterion(outputs, mids)
                outputs = autoencoder.decode1(outputs)
                loss.backward()
                loss2 = criterion(outputs, images)
                optimizer.step()
                running_loss += loss2.item() * images.size(0)
            epoch_loss = running_loss / len(dataset)
            print(f"Epoch [{epoch}/{EPOCHS}], Loss: {epoch_loss:.8f}")
        for epoch in range(1, EPOCHS+1):
            running_loss = 0.0
            for images in tqdm(dataloader):
                images = images.to(device)
                optimizer.zero_grad()
                mids = autoencoder.encode1(images)
                mids = autoencoder.encode2(mids)
                outputs = autoencoder.encode3(mids)
                outputs = autoencoder.decode3(outputs)
                loss = criterion(outputs, mids)
                loss.backward()
                outputs = autoencoder.decode2(outputs)
                outputs = autoencoder.decode1(outputs)
                loss2 = criterion(outputs, images)
                optimizer.step()
                running_loss += loss2.item() * images.size(0)
            epoch_loss = running_loss / len(dataset)
            print(f"Epoch [{epoch}/{EPOCHS}], Loss: {epoch_loss:.8f}")
        for epoch in range(1, EPOCHS+1):
            running_loss = 0.0
            for images in tqdm(dataloader):
                images = images.to(device)
                optimizer.zero_grad()
                outputs = autoencoder(images)
                loss = criterion(outputs, images)
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * images.size(0)
            epoch_loss = running_loss / len(dataset)
            print(f"Epoch [{epoch}/{EPOCHS}], Loss: {epoch_loss:.8f}")
    finally:
        torch.save(autoencoder.state_dict(), "autoencoder1.pth")
    return autoencoder

"""
def train(load = False):
    transform =  transforms.Compose([
        transforms.Resize((128,128)),
        transforms.ToTensor()
    ])
    dataset = ImageDataset(TRAIN_SET_PATH, transform)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    autoencoder = Autoencoder().to(device)
    if load:
        state_dict = torch.load("/kaggle/working/autoencoder.pth")
        autoencoder.load_state_dict(state_dict)
        
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(autoencoder.parameters(), lr=0.0001)
    try:
        for epoch in range(1, EPOCHS+1):
            running_loss = 0.0
            for images in tqdm(dataloader):
                images = images.to(device)
                optimizer.zero_grad()
                outputs = autoencoder(images)
                loss = criterion(outputs, images)
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * images.size(0)
            epoch_loss = running_loss / len(dataset)
            print(f"Epoch [{epoch}/{EPOCHS}], Loss: {epoch_loss:.8f}")
    finally:
        torch.save(autoencoder.state_dict(), "autoencoder.pth")
    return autoencoder

In [6]:
def getThreshold(FPrate = 0.1):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor()
    ])
    dataset = ImageDataset(TRAIN_SET_PATH, transform)
    dataloader = DataLoader(dataset, batch_size=1)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    autoencoder = Autoencoder().to(device)
    state_dict = torch.load("/kaggle/working/autoencoder1.pth")
    autoencoder.load_state_dict(state_dict)
    criterion = torch.nn.MSELoss()
    losses = []
    for images in dataloader:
        images = images.to(device)
        outputs = autoencoder(images)
        loss = criterion(outputs, images)
        losses.append(loss.item())
    losses = sorted(losses)
    print(-int(len(losses)*FPrate))
    return losses[-int(len(losses)*FPrate)]

In [7]:
def validate(threshold=0.004):
    transform =  transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor()
    ])
    dataset = ImageDataset(VAL_SET_PATH, transform)
    dataloader = DataLoader(dataset, batch_size=1)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    autoencoder = Autoencoder().to(device)
    state_dict = torch.load("/kaggle/working/autoencoder1.pth")
    autoencoder.load_state_dict(state_dict)
    criterion = torch.nn.MSELoss()
    abnormal = np.array([])
    for images in dataloader:
        images = images.to(device)
        outputs = autoencoder(images)
        loss = criterion(outputs, images)
        if loss.item() > threshold:
            abnormal = np.append(abnormal, 1)
        else:
            abnormal = np.append(abnormal, 0)
    return abnormal


In [10]:
autoencoder = train(False)

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

idx = 44
# print(thres)
# ae = Autoencoder().to(device)
# state_dict = torch.load("/kaggle/input/model-for-hc/autoencoder_part1.pth")
# ae.load_state_dict(state_dict)
dataset = ImageDataset(TRAIN_SET_PATH, transform)
transforms.ToPILImage()(dataset[idx]).save(f"origin{idx}.png")
img = dataset[idx].to("cuda")
images = img.view(1, 3, 128, 128)
outputs = autoencoder(images)
img = transforms.ToPILImage()(outputs[0])
img.save(f"test{idx}.png")

with open(VAL_TRUTH_PATH, 'r') as f:
    valtruths = f.readlines()
valtruths = np.array([int(valtruth[:-1]) for valtruth in valtruths])
thres = getThreshold(FPRATE)
print(thres)
abnormals = validate(thres)
FP = np.where(np.logical_and(abnormals==1,valtruths<=0),1,0)
TP = np.where(np.logical_and(abnormals==1,valtruths>=1),1,0)
FN = np.where(np.logical_and(abnormals==0,valtruths>=1),1,0)
TN = np.where(np.logical_and(abnormals==0,valtruths<=0),1,0)
Precision = np.sum(TP)/(np.sum(TP)+np.sum(FP))
Recall = np.sum(TP)/(np.sum(TP)+np.sum(FN))
F1_score = 2*Precision*Recall/(Precision+Recall)
FP_rate = np.sum(FP)/(np.sum(FP)+np.sum(TN))
print(np.sum(FP),np.sum(TP),np.sum(FN),np.sum(TN))
print("Precision:",Precision,"Recall:",Recall, "F1 Score:", F1_score, "FP Rate:", FP_rate)

100%|██████████| 13/13 [00:13<00:00,  1.07s/it]


Epoch [1/30], Loss: 0.06051075


100%|██████████| 13/13 [00:13<00:00,  1.04s/it]


Epoch [2/30], Loss: 0.03126010


100%|██████████| 13/13 [00:13<00:00,  1.06s/it]


...

Epoch [28/30], Loss: 0.00263352


100%|██████████| 13/13 [00:13<00:00,  1.05s/it]


Epoch [29/30], Loss: 0.00252392


100%|██████████| 13/13 [00:13<00:00,  1.05s/it]


Epoch [30/30], Loss: 0.00247219
-80
0.01448456384241581
27 47 121 53
Precision: 0.6351351351351351 Recall: 0.27976190476190477 F1 Score: 0.38842975206611574 FP Rate: 0.3375
